In [116]:
from database_io import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint


In [110]:
# path of databases (must exist)
db_path = "db/"

# filenames of databases (this must be sqlite3 databases)
train_fname = "surf17_train.db"
validation_fname = "surf17_validation.db"
test_fname = "surf17_test.db"

dim_syndr = 8
dim_fsyndr = 4
n_steps_net1 = 20
n_steps_net2 = 3

data = DatabaseIO(dim_syndr, dim_fsyndr, n_steps_net1, n_steps_net2)

In [120]:
try:
    data.close_databases()
except:
    pass
data.load_data(db_path + train_fname, db_path + validation_fname, db_path + test_fname)



batch_size = 64
n_batches_train = 500
n_batches_validation = 15



class DecoderSequence(Sequence):
    def __init__(self, data, batch_size, n_batches, data_type):
        self.data = data
        self.batch_size = batch_size
        self.n_batches = n_batches
        self.data_type = data_type
        self.on_epoch_end()

    def __len__(self):
        return self.n_batches

    def __getitem__(self, idx):
        # return the idx-th batch captured at epoch start
        return self.epoch_batches[idx]

    def on_epoch_end(self):
        """Called automatically by Keras at the end of each epoch."""
        gen = self.data.gen_batches(
            self.batch_size,
            self.n_batches,
            data_type=self.data_type
        )

        self.epoch_batches = []
        for _ in range(self.n_batches):
            batch_x1, batch_x2, batch_fx, batch_l1, batch_l2, batch_y = next(gen)

            # Wrap into Keras multi-input format
            inputs = (batch_x1, batch_x2, batch_fx) #, batch_l1, batch_l2)
            outputs = batch_y
            self.epoch_batches.append((inputs, outputs))

train_seq = DecoderSequence(
    data,
    batch_size=batch_size,
    n_batches=n_batches_train,
    data_type='training'
)

val_seq = DecoderSequence(
    data,
    batch_size=batch_size,
    n_batches=n_batches_validation,
    data_type='validation'
)


loaded databases and checked exclusiveness training, validation, and test keys
N_training=40000, N_validaiton=1000, N_test=5000.


In [126]:
# based on code from hands-on 5

# model = keras.models.Sequential(name="sequential_1")
# model.add(layers.LSTM(64, input_shape=(n_steps_net1,dim_syndr), return_sequences=True, name="lstm_1"))
# model.add(layers.LSTM(64, return_sequences=False, name="lstm_2"))
# model.add(layers.Flatten(name="flatten_1"))
# model.add(layers.Dropout(0.3, name="dropout_1"))
# model.add(layers.Dense(1, activation="sigmoid", name="dense_1"))

# model.summary()

# Input shapes from your data
# n_steps_net1 = T
# n_steps_net2 = T0
# dim_syndr    = syndrome dimension of δs⃗(t)
# fx input     = δf⃗(T) (shape = dim_syndr)


x1 = Input(shape=(None, dim_syndr), name="x1_full")
x1_masked = layers.Masking(mask_value=0.0)(x1)
x2 = Input(shape=(n_steps_net2, dim_syndr), name="x2_recent")
x2_masked = layers.Masking(mask_value=0.0)(x2)
fx = Input(shape=(dim_fsyndr,), name="final_increment")

# ---- Network 1 ---- (full syndrome history)
h1 = layers.LSTM(64, return_sequences=True)(x1_masked)
h1 = layers.Dropout(0.2)(h1)
h1 = layers.LSTM(64)(h1)
h1 = layers.Dropout(0.2)(h1)
p1 = layers.Dense(64, activation="relu", name="p1", kernel_regularizer=keras.regularizers.l2(1e-5))(h1)
p1 = layers.Dropout(0.2)(p1)
p1 = layers.Dense(1, activation="sigmoid", name="p1_prob")(p1)

# ---- Network 2 ---- (recent syndrome + final increment)
h2 = layers.LSTM(64, return_sequences=True)(x2_masked)
h2 = layers.Dropout(0.2)(h2)
h2 = layers.LSTM(64)(h2)
h2 = layers.Dropout(0.2)(h2)

# concatenate h2 with δf⃗(T)
h2_aug = layers.Concatenate()([h2, fx])

p2 = layers.Dense(64, activation="relu", name="p2", kernel_regularizer=keras.regularizers.l2(1e-5))(h2_aug)
p2 = layers.Dropout(0.2)(p2)
p2 = layers.Dense(1, activation="sigmoid", name="p2_prob")(p2)

# ---- Final combination p = probabilistic sum ---- #
p_final = layers.Lambda(lambda x: x[0]*(1-x[1]) + x[1]*(1-x[0]))([p1, p2])

model = Model(inputs=[x1, x2, fx], outputs=p_final)
model.summary()

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics=["accuracy"]
)


num_epochs = 100

checkpoint = ModelCheckpoint(
    'best_model.keras',       # file path to save the model
    monitor='val_accuracy',    # metric to monitor
    verbose=1,             # prints message when saving
    save_best_only=True,   # only save if improved
    mode='max'             # 'min' for loss, 'max' for accuracy
)

results = model.fit(
    train_seq,
    steps_per_epoch=n_batches_train,
    epochs=num_epochs,
    verbose=1,
    validation_data=val_seq,
    callbacks=[checkpoint]
)

Model: "functional_66"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ x2_recent           │ (None, 3, 8)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ x1_full             │ (None, None, 8)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_6         │ (None, 3, 8)      │          0 │ x2_recent[0][0]   │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_5         │ (None, None, 8)   │          0 │ x1_full[0][0]     │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_6 (Masking) │ (None, 3, 8)      │          0 │ x2_recent[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_6 (Any)         │ (None, 3)         │          0 │ not_equal_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_5 (Masking) │ (None, None, 8)   │          0 │ x1_full[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_5 (Any)         │ (None, None)      │          0 │ not_equal_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_106 (LSTM)     │ (None, 3, 64)     │     18,688 │ masking_6[0][0],  │
│                     │                   │            │ any_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_104 (LSTM)     │ (None, None, 64)  │     18,688 │ masking_5[0][0],  │
│                     │                   │            │ any_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_69          │ (None, 3, 64)     │          0 │ lstm_106[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_66          │ (None, None, 64)  │          0 │ lstm_104[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_107 (LSTM)     │ (None, 64)        │     33,024 │ dropout_69[0][0], │
│                     │                   │            │ any_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_105 (LSTM)     │ (None, 64)        │     33,024 │ dropout_66[0][0], │
│                     │                   │            │ any_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_70          │ (None, 64)        │          0 │ lstm_107[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_increment     │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_67          │ (None, 64)        │          0 │ lstm_105[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_42      │ (None, 68)        │          0 │ dropout_70[0][0], │
│ (Concatenate)       │                   │            │ final_increment[

 Total params: 112,130 (438.01 KB)

 Trainable params: 112,130 (438.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
499/500 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5588 - loss: 0.6650
Epoch 1: val_accuracy improved from None to 0.48750, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 22s 27ms/step - accuracy: 0.6211 - loss: 0.5992 - val_accuracy: 0.4875 - val_loss: 0.6970
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7539 - loss: 0.4189
Epoch 2: val_accuracy improved from 0.48750 to 0.53229, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.8015 - loss: 0.3735 - val_accuracy: 0.5323 - val_loss: 0.6846
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8850 - loss: 0.2790
Epoch 3: val_accuracy improved from 0.53229 to 0.59688, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.8888 - loss: 0.2628 - val_accuracy: 0.5969 - val_loss: 0.7321
Epoch 4/100
499/500 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9016 - loss: 0.2314
Epoch 4: val_accuracy impro

KeyboardInterrupt: 

In [133]:
num_samples = 500
for batch in data.gen_batches(num_samples, 1, data_type='validation'):
    errors = 0
    x1, x2, fx, _, _, y_actual = batch
    y_prob = model.predict((x1,x2,fx))
    for idx in range(y_actual.size):
        y_pred = y_prob[idx] < 0.5
        # print(f"Predicted Probability: {y_prob[idx]}  Prediction: {y_pred}  Actual: {y_actual[idx]}")
        if (y_pred != y_actual[idx]) :
            errors += 1
print(f"Accuracy: {errors / num_samples}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Accuracy: 0.664
